In [47]:
"""
JC Perrin
dernière modification 18/10/2021
"""
import os
import nmrglue as ng
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import *
from IPython.display import display
from scipy.signal import find_peaks
data_folder = 'C:/Users/perrin27/Documents/#LEMTA/POST-DOC_Aninda/traitement-CSI_experiments/'

In [289]:
def open_exp(num):
    num = ExpNum.value
#    ExpNum = 369 # valeur à lire widget
    decal_value = decal.value # valeur à lire widget
    
    dic, data = ng.bruker.read_pdata(data_folder + str(num) + "/pdata/1")
    udic = ng.bruker.guess_udic(dic, data, strip_fake=True)
# make ppm scales
    uc_13c = ng.fileiobase.uc_from_udic(udic, dim=1)
    ppm_13c = uc_13c.ppm_scale()
    # ppm_x = ppm_13c
    ppm_13c_0, ppm_13c_1 = uc_13c.ppm_limits()
    uc_15n = ng.fileiobase.uc_from_udic(udic, dim=0)
    ppm_15n = uc_15n.ppm_scale()
    
    # decalage pour centrer le spectre dans la dimension spatiale
    data_roll = np.roll(data, decal_value, axis = 0)
    return dic, data, data_roll, ppm_15n, ppm_13c, ppm_13c_0, ppm_13c_1


#________________________________________________________________________________#
    # Plot 2D contour plot
def plot_2d_contour():
    %matplotlib inline
    dic, data, data_roll, ppm_15n, ppm_13c, ppm_13c_0, ppm_13c_1 = open_exp(ExpNum)
    fig, ax = plt.subplots()
# TD1 = dic['acqus']['TD']
    TD2 = dic['acqu2s']['TD']
    ax.contour(data_roll, extent=(ppm_13c_0, ppm_13c_1, 0, TD2))
    plt.gca().invert_yaxis()
    plt.gca().invert_xaxis()
    ax.legend()
    plt.xlim([0, -17])
    plt.ylim([0, TD2])
    plt.xlabel('proton Chemical shift (ppm)')
    plt.ylabel('#point - spatial dimension (-)')


#________________________________________________________________________________#
    #Plot  projection along spatial dimension
def plot_1d_profil():
    %matplotlib inline
    num = ExpNum.value
#    ExpNum = 369 # valeur à lire depuis widget
    dic, data, data_roll, ppm_15n, ppm_13c, ppm_13c_0, ppm_13c_1 = open_exp(num)
    y_dist = np.arange(len(ppm_15n))
    dist_sum = np.sum(data_roll,axis=1)
    fig, ax = plt.subplots(figsize= [8, 4.8])
    ax.plot(y_dist,dist_sum, label = "ExpNum "+str(num))
    ax.scatter(y_dist,dist_sum)
#plt.xlim([0, ])
    plt.ylim([0, np.max(dist_sum)])
    plt.xlabel('#point - spatial dimension (-)')
    plt.ylabel('NMR intensity (-)')
#plt.title('UT paper 122019 _ #72 [2 layers 100%water / 60%w40%gly]',color='royalblue')
    text = 'projection along spatial dimension'
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    ax.text(0.05, 1.1, text, transform=ax.transAxes, fontsize=16,
            verticalalignment='top', bbox=props)
    plt.legend()          


#________________________________________________________________________________#
    #plot  projection along spectral dimension
def plot_proj_spectrum():
    %matplotlib inline
    dic, data, data_roll, ppm_15n, ppm_13c, ppm_13c_0, ppm_13c_1 = open_exp(ExpNum)
    ppm_sum = np.sum(data_roll,axis=0)
    fig, ax = plt.subplots(figsize= [8, 4.8])
    plt.plot(ppm_13c,ppm_sum)
    plt.xlim([15,-20])
    plt.ylim([0, 1.75e7])
    plt.xlabel('proton Chemical shift (ppm)')
    plt.ylabel('NMR intensity (-)')
#plt.title('UT paper 122019 _ #72 [2 layers 100%water / 60%w40%gly]',color='royalblue')
          
    text = 'projection along spectral dimension'
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    ax.text(0.05, 0.95, text, transform=ax.transAxes, fontsize=16,
            verticalalignment='top', bbox=props)

#________________________________________________________________________________#
    #plot  spectra at a specific location

def plot_spectrum():

    %matplotlib inline
    num = ExpNum.value
    point = loc.value
    dic, data, data_roll, ppm_15n, ppm_13c, ppm_13c_0, ppm_13c_1 = open_exp(num)
    fig, ax = plt.subplots(figsize= [8, 4.8])
    spectre = data_roll[point,:]
    plt.plot(ppm_13c,spectre,color = 'black')
    plt.xlim([10,-30])
    #plt.ylim([0, 10000])
    plt.xlabel('proton Chemical shift (ppm)')
    plt.ylabel('NMR intensity (-)')
    text = "ExpNum"+str(num) + ' -point# ' + str(point) 
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    ax.text(0.05, 0.95, text, transform=ax.transAxes, fontsize=16,
        verticalalignment='top', bbox=props)
    #fig.savefig('C:/Users/perrin27/Desktop/toto/CSI/'+ str(ExpNum) + '-point'+str(loc) +'.png')
    #plt.title(str(data_file))
    plt.show()

#________________________________________________________________________________#
    #calculate intensity ratio of the two resonance lines
    
    # find the maxima
def plot_Iratio():
    %matplotlib inline
    num = ExpNum.value
    point = loc.value
    dic, data, data_roll, ppm_15n, ppm_13c, ppm_13c_0, ppm_13c_1 = open_exp(num)
    fig, ax = plt.subplots(figsize= [8, 4.8])
    spectre = data_roll[point,:]
    plt.plot(ppm_13c,spectre,color = 'black')
    plt.xlim([10,-30])
    #plt.ylim([0, 10000])
    plt.xlabel('proton Chemical shift (ppm)')
    plt.ylabel('NMR intensity (-)')
    peaks, _ = find_peaks(spectre) # finds all local maxima by simple comparison of neighboring values 
    intensity_sorted = np.sort(spectre[peaks])[::-1]
    I_ratio = intensity_sorted[1]/intensity_sorted[0] # calculates the intensity ratio between the two first values of the "intensity_sorted" array
    plt.plot(ppm_13c[peaks],spectre[peaks], "x")
    text = "ExpNumum"+str(num) + ' -point# ' + str(point) +"\n I_ratio = {:.3f}".format(I_ratio)
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    ax.text(0.05, 0.95, text, transform=ax.transAxes, fontsize=16,
        verticalalignment='top', bbox=props)
    plt.show()
#     print(I_ratio)
#     return(I_ratio)

def calculate_Iratio(point):
    num = ExpNum.value
    dic, data, data_roll, ppm_15n, ppm_13c, ppm_13c_0, ppm_13c_1 = open_exp(num)
    spectre = data_roll[point,:]
    peaks, _ = find_peaks(spectre) # finds all local maxima by simple comparison of neighboring values 
    intensity_sorted = np.sort(spectre[peaks])[::-1]
    I_ratio = intensity_sorted[1]/intensity_sorted[0] # calculates the intensity ratio between the two first values of the "intensity_sorted" array
    return I_ratio
    
    
def Iratio_profile():
    start = range_slider.value[0]
    end = range_slider.value[1]
    pos = np.linspace(start-1, end-1, end-start+1, dtype = int)
    num = ExpNum.value
    Ir_profile = []
    for i in pos:
        I_ratio = calculate_Iratio(i)
        Ir_profile.append(I_ratio)
    fig, ax = plt.subplots(figsize= [8, 4.8])
    ax.plot(pos,Ir_profile, label = "ExpNum "+str(num))
    ax.scatter(pos,Ir_profile)
    plt.xlabel('#point - spatial dimension (-)')
    plt.ylabel('intensity ratio (-)')
    text = 'Intensity ratio of the two resonance lines'
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    ax.text(0.05, 1.1, text, transform=ax.transAxes, fontsize=16,
            verticalalignment='top', bbox=props)
    plt.legend()
        

In [290]:

calculate_Iratio(55)


0.9764245357359957

## creation of the widgets

In [295]:

"""
ExpNum value
"""
# ExpNum = widgets.IntSlider(
# value=369,
# min=0,
# max=1000,
# step=1,
# description='ExpNum:',
# disabled=False,
# continuous_update=False,
# orientation='horizontal',
# readout=True,
# readout_format='d'
# )

ExpNum = widgets.IntText(
    value=369,
    description='ExpNum:',
    disabled=False
)

"""
decal slider
"""
decal = widgets.IntSlider(
value=0,
min=-128,
max=128,
step=1,
description='decal:',
disabled=False,
continuous_update=False,
orientation='horizontal',
readout=True,
readout_format='d'
)

"""
plot_1d_profile BUTTON
callback = "plot_1d_profil()" function
"""
plot_1d_profile = widgets.Button(description="Plot 1D profile")
output = widgets.Output()
def on_button_clicked(b):
    with output:
        plot_1d_profil()
plot_1d_profile.on_click(on_button_clicked)

"""
plot_spectrum BUTTON
callback = "plot_spectrum()" function
"""
plot_spectr = widgets.Button(description="Plot spectrum")
output2 = widgets.Output()
def on_button_clicked2(b):
    with output2:
        plot_spectrum()
plot_spectr.on_click(on_button_clicked2)

"""
"Plot Iratio" BUTTON
callback = "calculate_Iratio()" function
calculates the intensity ratio of the two resonance lines = Imax(high_ppm_line)/Imax(low_ppm_line)
"""
Iratio = widgets.Button(description="Plot I ratio")
output3 = widgets.Output()
def on_button_clicked3(b):
    with output3:
        plot_Iratio()
Iratio.on_click(on_button_clicked3)


"""
range slider for Iratio profiles
"""
range_slider = widgets.IntRangeSlider(
    value=[30, 50],
    min=1,
    max=128,
    step=1,
    description='Range',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)

"""
Iratio profile BUTTON
"""

Iratio_prof = widgets.Button(description="Plot I ratio profile")
output4 = widgets.Output()
def on_button_clicked4(b):
    with output4:
        Iratio_profile()
Iratio_prof.on_click(on_button_clicked4)

In [296]:
display(ExpNum)
display(decal)
display(plot_1d_profile, output)
display(loc)
display(plot_spectr, output2)
display(Iratio, output3)
display(range_slider)
display(Iratio_prof, output4)

IntText(value=369, description='ExpNum:')

IntSlider(value=0, continuous_update=False, description='decal:', max=128, min=-128)

Button(description='Plot 1D profile', style=ButtonStyle())

Output()

IntText(value=90, description='#point')

...

Button(description='Plot I ratio', style=ButtonStyle())

Output()

IntRangeSlider(value=(30, 50), continuous_update=False, description='Range', max=128, min=1)

Button(description='Plot I ratio profile', style=ButtonStyle())

Output()

In [297]:
top_box = HBox([ExpNum, decal, loc, range_slider])
middle_box = HBox([plot_1d_profile,plot_spectr,Iratio,Iratio_prof])
bottom_box = HBox([output,output2,output3,output4])
VBox([top_box, middle_box,bottom_box])